In [0]:
# https://gist.github.com/korakot/5c8e21a5af63966d80a676af0ce15067
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!pip install selenium

%env PB_USER=YOUR_USERNAME
%env PB_PW=YOUR_PASSWORD

In [2]:
# https://gist.github.com/pybites/0aa6d9833849a0942ed218b1d46c47b4#file-platform_login_selenium-py
import os
from IPython.display import HTML
# make venv and pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# set your codechalleng.es username and password in venv/bin/activate, then source it
USERNAME = os.getenv("PB_USER") or "YOUR_USERNAME"
PASSWORD = os.getenv("PB_PW") or "YOUR_PASSWORD"
# Set your Bite to code
BITE_NUM = 104

BASE_URL = "https://codechalleng.es"
LOGIN_URL = f"{BASE_URL}/login/"
PYBITES_BITE_URL = f"{BASE_URL}/bites/{BITE_NUM}"
WAIT_SECONDS = 2
CODE_TEMPLATE = ""
TEST_NAME = ""
CODE_TEST = ""


# set options to be headless, ..
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver', options=options)

driver.get(LOGIN_URL)
  
driver.find_element_by_name('username').send_keys(USERNAME)
driver.find_element_by_name('password').send_keys(PASSWORD + Keys.RETURN)
  
# I am logged in, let go to a Bite
driver.get(PYBITES_BITE_URL)
# do your thing
#_html = driver.page_source
#print(_html)
contentWrapperElement = driver.find_element_by_css_selector("div.mui-col-md-8")
contentWrapperElement = contentWrapperElement.get_attribute("innerHTML")
# Implicit Wait
driver.implicitly_wait(WAIT_SECONDS)
element = driver.find_element_by_name("template_code")
CODE_TEMPLATE = element.get_attribute("value")
element = driver.find_element_by_xpath("//a[@data-mui-controls='pane-default-2']/span")
element.click()
TEST_NAME = element.text
TEST_NAME = TEST_NAME.strip("(").strip(")")
driver.execute_script("mui.tabs.activate('pane-default-2')")
# Simular Copy/Paste en Selenium
# Copy
cb = driver.execute_script("return copyTestToClipBoard();")
alert = driver.switch_to.alert
alert.accept()
driver.switch_to.default_content
# Crear textArea
js = """
function createTextArea(){
      var tArea = document.createElement('textarea');
      tArea.id = 'myTestCode';
      var pane = document.getElementById('pane-default-2');
      pane.appendChild(tArea);
}
createTextArea();
"""
cb = driver.execute_script(js)
# Paste
textAreaElement = driver.find_element_by_id("myTestCode")
textAreaElement.click()
textAreaElement.send_keys(Keys.CONTROL + "v")
CODE_TEST = textAreaElement.get_property("value")


BITE_NAME = TEST_NAME.split("test_")[1]

with open(BITE_NAME, "w") as file:
  file.write(CODE_TEMPLATE)
  os.environ["PB_FILE"] = BITE_NAME

with open(TEST_NAME, "w") as file:
  file.write(CODE_TEST)
  os.environ["PB_TEST"] = TEST_NAME

_html = contentWrapperElement + "\n\n" + \
f"<b>{BITE_NAME} and {TEST_NAME} files written to local storage!</b>"

HTML(_html)


In [8]:
!cat $PB_FILE

message = """Hello world!
We hope that you are learning a lot of Python.
Have fun with our Bites of Py.
Keep calm and code in Python!
Become a PyBites ninja!"""

def split_in_columns(message=message):
  return "|".join(message.split("\n"))

Copy and paste the Code Template into the next cell. Write the solution and overwrite it to the file using the magic command **%%writefile \<PB_FILE\>** in the first line of the cell

In [4]:
print(f"%%writefile {BITE_NAME}")

%%writefile pipe.py


In [5]:
%%writefile pipe.py
message = """Hello world!
We hope that you are learning a lot of Python.
Have fun with our Bites of Py.
Keep calm and code in Python!
Become a PyBites ninja!"""

def split_in_columns(message=message):
  return "|".join(message.split("\n"))


Overwriting pipe.py


Uncomment the next line and run it until passing tests with **pytest**

In [6]:
#!pytest [options] $PB_TEST
!pytest --capture=fd $PB_TEST
print(f"!python -m pytest {TEST_NAME}")

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 2 items                                                              

test_pipe.py ..                                                          [100%]

=========================== 2 passed in 0.02 seconds ===========================
!python -m pytest test_pipe.py


When you are sure that your code passes the Test, execute the next cell for **”SAVE AND RUN TESTS**

In [18]:
with open(BITE_NAME, "rb") as file:
  CODE = file.read()

CODE = str(CODE).strip("b").strip("'")

js = f"""function MyCopyCodeToForm(){{
  var code = `{CODE}`;
  document.getElementById('code').value = code;
}}
MyCopyCodeToForm();
var saveRunTest = document.getElementById('save');
saveRunTest.setAttribute('onclick', 'return MyCopyCodeToForm();')
saveRunTest.click();
"""
driver.execute_script(js)
print("SAVE AND RUN TESTS")

SAVE AND RUN TESTS
